In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, f1_score, precision_recall_fscore_support,
    hamming_loss, jaccard_score, roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve, confusion_matrix
)

from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
df = pd.read_csv("database.csv") 
mapping = {'L': 0, 'M': 1, 'H': 2}
df['Type'] = df['Type'].map(mapping)

X = df[['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', "Tool wear [min]"]] 
Y = df[["TWF", "HDF", "PWF", "OSF", "RNF"]] 
X_np=X.values
Y_np=Y.values

In [ ]:
X_train_temp, Y_train_temp, X_test, Y_test = iterative_train_test_split(X_np, Y_np, test_size=0.3) 
X_train, Y_train, X_val, Y_val = iterative_train_test_split(X_train_temp, Y_train_temp, test_size=0.3)
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
X_val_s= scaler.transform(X_val)

In [ ]:
# Modelo
def build_mlp(input_dim, n_labels):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(n_labels, activation='sigmoid')
    ])
    return model

# Metrica F1 
class F1Score(keras.metrics.Metric):
    def __init__(self, name='f1_score', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.precision = keras.metrics.Precision(thresholds=threshold)
        self.recall = keras.metrics.Recall(thresholds=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_bin = tf.cast(y_pred > self.threshold, tf.float32)
        self.precision.update_state(y_true, y_pred_bin, sample_weight=sample_weight)
        self.recall.update_state(y_true, y_pred_bin, sample_weight=sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + 1e-7))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

    def get_config(self):
        return {"threshold": float(self.threshold)}

def compile_model(model, lr=1e-3):
    metrics = [
        keras.metrics.BinaryAccuracy(name='binary_accuracy', threshold=0.5),
        keras.metrics.Precision(name='precision', thresholds=0.5),
        keras.metrics.Recall(name='recall', thresholds=0.5),
        F1Score(name='f1_score', threshold=0.5)
    ]
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=metrics
    )
    return model

mlp = build_mlp(input_dim=X_train_s.shape[1], n_labels=Y_train.shape[1])
mlp = compile_model(mlp, lr=1e-3)

es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

history = mlp.fit(
    X_train_s, Y_train,
    validation_data=(X_val_s, Y_val),
    epochs=70,
    batch_size=16,
    callbacks=[es, rlr],
    verbose=2
)

y_prob = mlp.predict(X_test_s)            
y_pred_default = (y_prob > 0.5).astype(int)

In [ ]:
# Matrices de confusion por etiqueta
labels=["TWF", "HDF", "PWF", "OSF", "RNF"]
for i, lab in enumerate(labels):
    cm = confusion_matrix(Y_test[:,i], y_pred_default[:,i])
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion matrix {lab}")
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.show()